Encoders:
 - https://huggingface.co/monsoon-nlp/hindi-bert
 - https://huggingface.co/flax-community/roberta-hindi
 - https://huggingface.co/vasista22/whisper-hindi-small
 
Decoders:
- https://huggingface.co/csebuetnlp/mT5_m2o_hindi_crossSum
- https://huggingface.co/docs/transformers/model_doc/openai-gpt
- https://huggingface.co/docs/transformers/model_doc/ctrl
- https://huggingface.co/docs/transformers/model_doc/gpt2

In [82]:
# from transformers import EncoderDecoderModel, AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("monsoon-nlp/hindi-bert")
# model = EncoderDecoderModel.from_encoder_decoder_pretrained("monsoon-nlp/hindi-bert", "gpt2")

# model.config.decoder_start_token_id = tokenizer.cls_token_id
# model.config.pad_token_id = tokenizer.pad_token_id

# # a hindi sentence
# input_ids = tokenizer(
#     'भारतीय राष्ट्रीय कांग्रेस के अध्यक्ष राहुल गांधी ने शनिवार को कहा कि भारतीय जनता पार्टी ने देश को एक दशक में वापस ले जाया है।',
#     return_tensors="pt",
# ).input_ids

# labels = tokenizer(
#     'राहुल गांधी ने कहा कि भारतीय जनता पार्टी ने देश को एक दशक में वापस ले जाया है।', return_tensors="pt",
# ).input_ids

# # the forward function automatically creates the correct decoder_input_ids
# outputs = model(input_ids=input_ids, labels=labels)

In [83]:
# outputs = model.generate(input_ids, decoder_start_token_id=model.config.decoder_start_token_id)
# outputs

In [84]:
# tokenizer2 = AutoTokenizer.from_pretrained("gpt2")
# tokenizer2.decode(outputs[0], skip_special_tokens=True, predict_with_generate=True)

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

test_df = pd.read_csv('../Data/HindiNews_test.csv')
train_df = pd.read_csv('../Data/hindi_train.csv')
print(train_df.shape, test_df.shape)

train_df = train_df.dropna()
test_df = test_df.dropna()

train_val_split = 0.15
train_df, val_df = train_test_split(train_df, test_size=train_val_split, random_state=42)
print(train_df.shape, val_df.shape, test_df.shape)

(21225, 4) (3000, 3)
(18041, 4) (3184, 4) (3000, 3)


In [3]:
train_df.head()

,Id,Heading,Summary,Article
8713,hindi_2023_train_8713,प्रोडक्शन के लिए एम्ब्रेयर और सुखोई की बातचीत ...,india looks partner sukhoi embraer to manufact...,भारत सरकार दूर-दराज इलाकों से बेहतर एयर कनेक्ट...
19043,hindi_2022_9186,UP: भदोही में नशे में धुत युवक ने महात्मा गांध...,जिसके बाद कांग्रेस कार्यकर्ता एकत्रित होकर गां...,भदोहीः उत्तर प्रदेश के भदोही में बीती शनिवार र...
17257,hindi_2022_7400,भारत के इस क्षेत्र में नहीं है Coronavirus का ...,भारत में कोविड-19 के मामले 97.35 लाख के पार चल...,कोच्चि: भारत में कोविड-19 के मामले 97.35 लाख क...
3156,hindi_2023_train_3156,12वीं में पासिंग परसेंटेज के आधार पर परीक्षा म...,NEET 2024 - No Minimum Marks Needed For NEET. ...,12वीं के बाद मेडिकल की पढ़ाई करने वाले छात्रों ...
15583,hindi_2022_5726,"उत्तर प्रदेश में कोरोना वायरस से 17 और मौतें, ...",उत्‍तर प्रदेश में शुक्रवार को कोरोना वायरस संक...,लखनऊ: उत्‍तर प्रदेश में शुक्रवार को कोरोना वाय...


In [83]:
import torch
from transformers import EncoderDecoderModel, AutoTokenizer, AutoModelForSeq2SeqLM, MT5ForConditionalGeneration


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer_mT5_m2o = AutoTokenizer.from_pretrained("csebuetnlp/mT5_m2o_hindi_crossSum")
decoder_model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/mT5_m2o_hindi_crossSum")
#decoder_model.enc_to_dec_proj=torch.nn.Linear(768, len(tokenizer_indicbart))
#decoder_model.enc_to_dec_proj.name = "lm_head"

tokenizer_indicbart = AutoTokenizer.from_pretrained("ai4bharat/IndicBART") #"aashay96/indic-gpt"
encoder_model =  AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/IndicBART")

model = EncoderDecoderModel.from_encoder_decoder_pretrained(encoder_model=encoder_model.get_encoder(), decoder_model=decoder_model.get_decoder())
model.config.decoder_start_token_id = tokenizer_indicbart.cls_token_id
model.config.pad_token_id = tokenizer_indicbart.pad_token_id
model = model.to(device)


/mnt/disk1/sumdev/CourseWork/a3_env/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [84]:
model

EncoderDecoderModel(
  (encoder): MBartEncoder(
    (embed_tokens): Embedding(64014, 1024, padding_idx=0)
    (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
    (layers): ModuleList(
      (0-5): 6 x MBartEncoderLayer(
        (self_attn): MBartAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (activation_fn): GELUActivation()
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      )
    )
    (layernorm_embedding): LayerNorm((1024,), eps=

In [106]:
encoder_model2 = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/IndicBART")
decoder_model2 = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/mT5_m2o_hindi_crossSum")
# decoder_model2.encoder = encoder_model.get_encoder()

In [115]:
decoder_model2

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
         

In [110]:
encoder_model2

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(64014, 1024, padding_idx=0)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(64014, 1024, padding_idx=0)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-5): 6 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): La

In [127]:
import torch.nn as nn

fc = nn.Linear(1024, 768)
x = torch.randn(1, 29, 1024)
output = fc(x)
output.shape

torch.Size([1, 29, 768])

In [1]:
# I want to pass a tokenizer output to the encoder_model2 model's encoder only
import torch
x = torch.tensor(tokenizer_indicbart.encode("भारतीय राष्ट्रीय कांग्रेस के अध्यक्ष राहुल गांधी ने शनिवार को कहा कि भारतीय जनता पार्टी ने देश को एक दशक में वापस ले जाया ह।", return_tensors="pt"))
print(x.shape)
out = encoder_model2(x)
print(out.encoder_last_hidden_state.shape)
x = torch.tensor(tokenizer_mT5_m2o.encode("भारतीय राष्ट्रीय कांग्रेस के अध्यक्ष राहुल गांधी ने शनिवार को कहा कि भारतीय जनता पार्टी ने देश को एक दशक में वापस ले जाया ह।", return_tensors="pt"))
print(x.shape)
print(x)

out2 = decoder_model2.decoder(x)
# print(out2)
print(out2.last_hidden_state.shape) 

NameError: name 'tokenizer_indicbart' is not defined

In [88]:
encoder_model2

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(64014, 1024, padding_idx=0)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(64014, 1024, padding_idx=0)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-5): 6 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): La

In [89]:
decoder_model2

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
         

In [20]:
model.decoder.resize_token_embeddings(len(tokenizer))
#model.encoder.resize_token_embeddings(len(tokenizer))


Embedding(64014, 768)

In [22]:
model.decoder

MT5Stack(
  (embed_tokens): Embedding(64014, 768)
  (block): ModuleList(
    (0): MT5Block(
      (layer): ModuleList(
        (0): MT5LayerSelfAttention(
          (SelfAttention): MT5Attention(
            (q): Linear(in_features=768, out_features=768, bias=False)
            (k): Linear(in_features=768, out_features=768, bias=False)
            (v): Linear(in_features=768, out_features=768, bias=False)
            (o): Linear(in_features=768, out_features=768, bias=False)
            (relative_attention_bias): Embedding(32, 12)
          )
          (layer_norm): MT5LayerNorm()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (1): MT5LayerCrossAttention(
          (EncDecAttention): MT5Attention(
            (q): Linear(in_features=768, out_features=768, bias=False)
            (k): Linear(in_features=768, out_features=768, bias=False)
            (v): Linear(in_features=768, out_features=768, bias=False)
            (o): Linear(in_features=768, out_features=768,

In [4]:
import torch
model_indic.config.pad_token_id = tokenizer.pad_token_id

device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
model_indic = model_indic.to(device)


In [6]:

import torch.nn as nn
import torch.optim as optim
model_indic.resize_token_embeddings(len(tokenizer))
optimizer = optim.Adam(model_indic.parameters(), lr=1e-3)
num_epochs = 3

NameError: name 'model_indic' is not defined

In [14]:
import torch.nn as nn
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=1e-3)
num_epochs = 3

In [22]:
def train(model, tokenizer, train_df, optimizer):
    model.train()
    total_loss = 0
    batch = 0
    batch_loss = 0
    batch_size = 1000
    for i, row in train_df.iterrows():
        input = tokenizer(row['Heading'], return_tensors="pt")
        input_ids = input.input_ids.to(device)
        attention_mask = input.attention_mask.to(device)
        labels = tokenizer(row['Summary'], return_tensors="pt")
        labels_input_ids = labels.input_ids.to(device)
        labels_attention_mask = labels.attention_mask.to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask,  labels=labels_input_ids, decoder_attention_mask=labels_attention_mask)
        loss = outputs.loss
        total_loss += loss.item()
        batch_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        batch += 1
        if batch % batch_size == 0:
            print(f"Batch {batch} Loss: {batch_loss / batch_size}")
            batch_loss = 0
    print(f"Training Loss: {total_loss / len(train_df)}")
        

def evaluate(model, tokenizer, val_df):
    model.eval()
    total_loss = 0
    for i, row in val_df.iterrows():
        input = tokenizer(row['Heading'], return_tensors="pt")
        input_ids = input.input_ids.to(device)
        attention_mask = input.attention_mask.to(device)
        labels = tokenizer(row['Summary'], return_tensors="pt")
        labels_input_ids = labels.input_ids.to(device)
        labels_attention_mask = labels.attention_mask.to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask,  labels=labels_input_ids, decoder_attention_mask=labels_attention_mask)
        total_loss += outputs.loss.item()
    print(f"Validation Loss: {total_loss / len(val_df)}")


for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}")
    train(model, tokenizer, train_df, optimizer)
    evaluate(model, tokenizer, val_df)

Epoch 1


/raid/home/dhruv/miniconda3/envs/myenv/lib/python3.11/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:636: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


AttributeError: 'BaseModelOutputWithPastAndCrossAttentions' object has no attribute 'logits'

In [24]:
text = test_df['Article'][6]
prompt="Please give me text summary of following hindi articlein hindi:"
print(test_df['id'][6])
print(text)
input_ids = tokenizer_indicbart(prompt+text, return_tensors="pt", truncation=True).input_ids.to(device)
print(input_ids.shape)

outputs = model.generate(input_ids, decoder_start_token_id=model.config.decoder_start_token_id, num_beams=4,max_new_tokens=100, temperature = 0.4, do_sample=True)
print(outputs)
# tokenizer2
summary = tokenizer_mT5_m2o.decode(outputs[0], skip_special_tokens=True, predict_with_generate=True)
print("Summary:", summary)

HindiNews_test_6
साल-दर-साल बढ़ती गर्मी के चलते देश की खेती, अर्थव्यवस्था और लोगों के स्वास्थ्य पर बहुत दबाव पड़ रहा है। वहीं, जलवायु परिवर्तन की वजह से गरीबी, असमानता और बीमारियों को खत्म करने के देश की कोशिशें भी नाकाम हो रही हैं। यह खुलासा कैंब्रिज यूनिवर्सिटी की एक नई स्टडी में हुआ है। इसे स्कॉलर्स की एक टीम ने पब्लिश किया है।
टीम के लीडर रमित देबनाथ ने कहा कि लू के चलते 1992 से अब तक देश में 24 हजार से ज्यादा मौतें दर्ज हुई हैं। गर्मी के चलते वायु प्रदूषण बढ़ गया है और उत्तर भारत के पहाड़ों में मौजूद ग्लेशियर पिघलने लगे हैं। देश में जनवरी से अक्टूबर तक तकरीबन हर रोज भीषण मौसम देखने को मिल रहा है, जिसकी वजह से देश को एक साथ कई सारी मौसमी परेशानियों का सामना करना पड़ सकता है।
देश का 90% इलाका हीट जोन बना 
देबनाथ ने रॉयटर्स को बताया कि लगभग भारत के कुल एरिया में से 90% गंभीर हीट जोन बन चुके हैं और ये इस गर्मी से निपटने से के लिए तैयार नहीं हैं। उन्होंने कहा कि भारत ने लू को अपने डिजास्टर रिलीफ पैकेज में शामिल किया है, लेकिन इन योजनाओं की रफ्तार को तेज करने की जरूरत है।
भीषण गर्मी के चलते 

NotImplementedError: A model class needs to define a `prepare_inputs_for_generation` method in order to use `.generate()`.